<a href="https://colab.research.google.com/github/ZacharyZekaiXu/ZekaiXu_CrossmodalRecognition/blob/main/ZX_BeeCrossmodalTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bee_Tactile-Visual_Crossmodality_RL-Task

Zekai Xu, Impeiral College

### I. Reference

### II. Updates and To-Do-List







# Install, Inputs

In [ ]:
#@title Run to install MuJoCo and `dm_control`
import distutils.util
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

print('Installing dm_control...')
!pip install -q dm_control>=1.0.5

# Configure dm_control to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

print('Checking that the dm_control installation succeeded...')
try:
  from dm_control import suite
  env = suite.load('cartpole', 'swingup')
  pixels = env.physics.render()
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')
else:
  del pixels, suite

!echo Installed dm_control $(pip show dm_control | grep -Po "(?<=Version: ).+")

Installing dm_control...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.5 which is incompatible.
tensorflow-metadata 1.9.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.5 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.5 which is incompatible.
google-api-core 1.31.6 requires protobuf<4.0.0dev,>=3.12.0; python_version > "3", but you have protobuf 4.21.5 which is incompatible.
env: MUJOCO_GL=egl
Checking that the dm_control installation succeeded...
Installed dm_control 1.0.5


In [ ]:
#@title All `dm_control` imports required for this tutorial

# The basic mujoco wrapper.
from dm_control import mujoco

# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib

# PyMJCF
from dm_control import mjcf

# Composer high level imports
from dm_control import composer
from dm_control.composer.observation import observable
from dm_control.composer import variation

# Imports for Composer tutorial example
from dm_control.composer.variation import distributions
from dm_control.composer.variation import noises
from dm_control.locomotion.arenas import floors

# Control Suite
from dm_control import suite

# Run through corridor example
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.arenas import corridors as corridor_arenas
from dm_control.locomotion.tasks import corridors as corridor_tasks

# Soccer
from dm_control.locomotion import soccer

# Manipulation
from dm_control import manipulation

## ACME

In [1]:
!pip uninstall sonnet
!pip install dm-sonnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#@title Install necessary dependencies.

!sudo apt-get install -y xvfb ffmpeg
!pip install 'gym==0.10.11'
!pip install imageio
!pip install PILLOW
!pip install 'pyglet==1.3.2'
!pip install pyvirtualdisplay

!pip install dm-acme
!pip install dm-acme[reverb]
!pip install dm-acme[tf]

from IPython.display import clear_output
clear_output()

In [3]:
!pip uninstall dm-acme[envs]

Found existing installation: dm-acme 0.4.0
Uninstalling dm-acme-0.4.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/acme/*
    /usr/local/lib/python3.7/dist-packages/dm_acme-0.4.0.dist-info/*
Proceed (y/n)?   ERROR: Operation cancelled by user


In [4]:
#@title Import modules.
#python3

%%capture
import copy
import pyvirtualdisplay
import imageio 
import base64
import IPython

from acme import environment_loop
from acme.tf import networks
from acme.adders import reverb as adders
from acme.agents.tf import actors as actors
from acme.datasets import reverb as datasets
from acme.wrappers import gym_wrapper
from acme import specs
from acme import wrappers
from acme.agents.tf import d4pg
from acme.agents.tf import ddpg
from acme.agents import agent
from acme.tf import utils as tf2_utils
from acme.utils import loggers

import gym 

import matplotlib.pyplot as plt
import numpy as np
import reverb
import sonnet as snt
import tensorflow as tf

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

## Mujoco

In [5]:
!pip install mujoco-py
!pip install xmltodict
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf
clear_output()

In [6]:
!python --version

Python 3.7.13


In [6]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py
import mujoco_py
import gym

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglew-dev is already the newest version (2.0.0-5).
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
libosmesa6-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
patchelf is already the newest version (0.9-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Looking in indexe

## Google Drive

In [13]:
from google.colab import drive
drive.mount('/content/zxdrive')

%cd zxdrive/MyDrive/BeeCrossmodalityPJ/safety-gym
!git pull

Drive already mounted at /content/zxdrive; to attempt to forcibly remount, call drive.mount("/content/zxdrive", force_remount=True).
/content/zxdrive/MyDrive/BeeCrossmodalityPJ/safety-gym
Already up to date.


## dm_control

## Imports

Run both of these cells:

In [18]:
#@title Other imports and helper functions

# General
import os
import copy
import itertools
from IPython.display import clear_output
import numpy as np

# Graphics-related
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image
# Internal loading of video libraries.

# Use svg backend for figure rendering
%config InlineBackend.figure_format = 'svg'

# Font sizes
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Inline video helper function
if os.environ.get('COLAB_NOTEBOOK_TEST', False):
  # We skip video generation during tests, as it is quite expensive.
  display_video = lambda *args, **kwargs: None
else:
  def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Seed numpy's global RNG so that cell outputs are deterministic. We also try to
# use RandomState instances that are local to a single cell wherever possible.
np.random.seed(42)

# Model definition, compilation and rendering



Here are some basic concepts of the [MuJoCo](http://mujoco.org/) physics simulation library, recommend the [official documentation](http://mujoco.org/book/) for details.

### Model Settings

In [19]:

import mujoco_py


bee_pointmodel = """
<mujoco>
    <size njmax="3000" nconmax="1000"/>
    <option timestep="0.002"/>
    <default>
        <geom condim="6" density="1" rgba="1 1 0 1"/>
        <joint damping=".001"/>
        <motor ctrlrange="-1 1" ctrllimited="true" forcerange="-.05 .05" forcelimited="true"/>
        <velocity ctrlrange="-1 1" ctrllimited="true" forcerange="-.05 .05" forcelimited="true"/>
        <site size="0.032" type="sphere"/>
    </default>
    <worldbody>-
        <light name="toplight" pos="0 0 1"/>
        <geom name="floor" size="5 5 0.1" type="plane" condim="6"/>
        <body name="robot" pos="0 0 .1">
            <camera name="vision1" pos="0 0 .15" xyaxes="0 -1 0 .4 0 1" fovy="90"/>
            <!-- <geom name="camera" pos="-.2 0 0" type="sphere" size=".05"/> -->
            <joint type="slide" axis="1 0 0" name="x" damping="0.01"/>
            <joint type="slide" axis="0 1 0" name="y" damping="0.01"/>
            <joint type="hinge" axis="0 0 1" name="z" damping="0.005"/>
            <geom name="robot" type="sphere" size=".1" friction="1 0.01 0.01"/>
            <geom name="beehead" pos="0.09 0 0" size="0.06" type="sphere" density = "0"/>
            <geom name="beewhister1" type="cylinder"  size="0.004 .8" fromto="0.1 -.03 0 0.5 -.03 0"/>
            <geom name="beewhister2" type="cylinder"  size="0.004 .8" fromto="0.1 .03 0 0.5 .03 0"/>
            <geom name="beeeye1" type="sphere" size=".02" pos=".11 -.03 0.05" rgba="0 0 0 1"/>
            <geom name="beeeye2" type="sphere" size=".02" pos=".11 0.03 0.05" rgba = "0 0 0 1"/>
            <site name="robot" rgba="1 0 0 .1"/>
        </body>
    </worldbody>
    <sensor>
        <!-- Used for observation -->
        <accelerometer site="robot" name="accelerometer"/>
        <velocimeter site="robot" name="velocimeter"/>
        <gyro site="robot" name="gyro"/>
        <magnetometer site="robot" name="magnetometer"/>
        <!-- Used for intrinsic constraints -->
        <subtreecom body="robot" name="subtreecom"/>
        <subtreelinvel body="robot" name="subtreelinvel"/>
        <subtreeangmom body="robot" name="subtreeangmom"/>
    </sensor>
    <actuator>
        <motor gear="0.3 0 0 0 0 0" site="robot" name="x"/>
        <velocity gear="0.3" jointinparent="z" name="z"/>
    </actuator>
</mujoco>
"""

physics = mujoco.Physics.from_xml_string(bee_pointmodel)
pixels = physics.render()
PIL.Image.fromarray(pixels)

NameError: ignored

`static_model` is written in MuJoCo's XML-based [MJCF](http://www.mujoco.org/book/modeling.html) modeling language. The `from_xml_string()` method invokes the model compiler, which instantiates the library's internal data structures. These can be accessed via the `physics` object, see below.

In [9]:
#@title Making a video {vertical-output: true}

duration = 5    # (seconds)
framerate = 30  # (Hz)

# Visualize the joint axis
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.flags[enums.mjtVisFlag.mjVIS_JOINT] = False

# Simulate and display video.
frames = []
physics.reset()  # Reset state and time
while physics.data.time < duration:
  physics.step()
  if len(frames) < physics.data.time * framerate:
    pixels = physics.render(scene_option=scene_option)
    frames.append(pixels)
display_video(frames, framerate)

Note how we collect the video frames. Because physics simulation timesteps are generally much smaller than framerates (the default timestep is 2ms), we don't render after each step.

### Rendering options

additional rendering options are exposed as parameters to the `render` method.

In [11]:
#@title Depth rendering {vertical-output: true}

# depth is a float array, in meters.
depth = physics.render(depth=True)
# Shift nearest values to the origin.
depth -= depth.min()
# Scale by 2 mean distances of near rays.
depth /= 2*depth[depth <= 1].mean()
# Scale to [0, 255]
pixels = 255*np.clip(depth, 0, 1)
PIL.Image.fromarray(pixels.astype(np.uint8))

NameError: ignored

## OpenAI Safety Gym Custom

In [14]:
!pip install git+https://github.com/openai/safety-gym.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/safety-gym.git to /tmp/pip-req-build-_b_nsjdn
  Running command git clone -q https://github.com/openai/safety-gym.git /tmp/pip-req-build-_b_nsjdn
  Using cached gym-0.15.7-py3-none-any.whl
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
  Using cached mujoco-py-2.0.2.7.tar.gz (777 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached numpy-1.17.5-cp37-cp37m-manylinux1_x86_64.whl (20.0 MB)
  Using cached xmltodict-0.12.0-py2.py3-none-any.whl (9.2 kB)
  Using cached cloudpickle-1.2.2-py2.py3-none-any.whl (25 kB)
  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Created wheel for safety-gym: filename=safety_gym-0.0.0-py3-none-any.whl size=2722 sha256=c2abea0506c38f8b1c9443fedf1e9a92e8cae01a56dd7d823b8d88

In [15]:
import safety_gym # /content/zxdrive/MyDrive/BeeCrossmodalityPJ/safety-gym/safety_gym
print(safety_gym)

<module 'safety_gym' from '/content/zxdrive/MyDrive/BeeCrossmodalityPJ/safety-gym/safety_gym/__init__.py'>


In [16]:
import gym

env = gym.make('Safexp-PointGoal1-v0')
from safety_gym import 

In [21]:
from safety_gym import engine

ImportError: ignored

In [18]:
frame = render(env)
plt.imshow(frame)
plt.axis('off')

NameError: ignored

## Setting the state with `reset_context()`

In order for `data` quantities that are functions of the state to be in sync with the state, MuJoCo's `mj_step1()` needs to be called. This is facilitated by the `reset_context()` context, please see in-depth discussion in Section 2.1 of the [tech report](https://arxiv.org/abs/2006.12983).

# RL Agent DDPG

In [20]:
!pip install graphs
clear_output()

In [8]:
def make_networks_ddpg(action_spec,
                       policy_layer_sizes=(256, 256, 256),
                       critic_layer_sizes=(512, 512, 256),
                      ):

  """Networks for DDPG agent."""
  action_size = np.prod(action_spec.shape, dtype=int)

  policy_network = snt.Sequential([
      tf2_utils.batch_concat,
      networks.LayerNormMLP(layer_sizes=policy_layer_sizes + (action_size,)),
      networks.TanhToSpec(spec=action_spec)
      ])
  critic_network = networks.CriticMultiplexer(
          action_network=networks.ClipToSpec(action_spec),
          critic_network=networks.LayerNormMLP(
              layer_sizes=critic_layer_sizes + (1,),
              activate_final=False),
              )

  return policy_network, critic_network

### Functions for saving and restoring checkpoints

Due to Colab usage limits, the Colab runtime will be restarting periodically. In order to preserve the most recent training checkpoint during a restart, please use the functions below.

To do so, you'll have to first mount Google Drive (will be shown below).

Before runtime restart:
Use save_ckpt_to_drive to locate the checkpoint and save it to your Google Drive in a directory /acme_ckpt

After runtime restart:
Use restore_ckpt_from_drive to recover the checkpoint from Google Drive and copy it back to the restarted Colab virtual machine.